In [11]:
import pandas as pd

In [12]:
#データ読み込み&表示
#df = pd.read_csv('data/allgame_result.csv')
df = pd.read_csv('data_new/allgame_result_corrected.csv')
display(df.head())
length = df.shape[0]
print(length)

,投球数,球種,球速,結果,投手,打者,スコア,アウトカウント
0,1,ストレート,147km/h,中飛,菅野 智之,大島 洋平,0-0,1
1,1,ストレート,144km/h,ボール,菅野 智之,岡林 勇希,0-0,2
2,2,カットボール,139km/h,見逃し,菅野 智之,岡林 勇希,0-0,2
3,3,スライダー,132km/h,二ゴロ,菅野 智之,岡林 勇希,0-0,2
4,1,ツーシーム,147km/h,ボール,菅野 智之,福留 孝介,0-0,3


243709


In [13]:
#リプレー検証の該当プレーの数を確認
cnt = 0
for index,data in df.iterrows():
    if ('リプレー検証' in data['結果']) or ('リプレー検証後判定変わらず' in data['結果']) or ('リプレー検証後判定覆る' in data['結果']):
        #print(data['結果'],index)
        cnt += 1
print(cnt)        

575


In [14]:
#リプレー検証直後のプレイを配列にして取り出したもの
#arrayにはリプレー検証直後のプレー
#リプレー検証でイニングや試合が終わっていれば直後のプレーは取りださない
cnt = 0
array = []
length = df.shape[0]
for index,data in df.iterrows():
    if ('リプレー検証' in data['結果']) or ('リプレー検証後判定変わらず' in data['結果']) or ('リプレー検証後判定覆る' in data['結果']):
        cnt += 1
        
        #リプレイ検証に関わるプレーでイニングが終了したときの処理
        if df.iat[index,7] == 3: #前のプレーのアウトカウントが3なら(次のイニングに行ったなら):
            continue
        
        prev = 0
        now = 0
        
        for i in range(index+1,length):
            prev = df.iat[i, 0]
            if i==length-1:
                array.append(df.iat[i,3])
                break
            
            #投球数を見てプレーの切れ目を確認する
            else:
                now = df.iat[i+1,0]
                if prev >= now:
                    array.append(df.iat[i,3])
                    break
        
print(array)
print('リプレー検証に関わるプレーは'+str(cnt)+'箇所')
print('リプレー検証直後のプレーの合計は'+str(len(array))+'箇所')

['中２  [フェンス直撃]', '遊ゴロ', '左２  [泳ぎ、バットの先、内野一塁寄り]', '四球  [ワンバウンド、ランエンドヒット]', '左飛  [高々と舞い上がる]', '投犠打', '中飛', '四球', '三邪飛', '一ゴロ  [走者動けず]', '中２  [目測を誤る]', '右２', '中２', '空三振', '遊ゴロ  [止めたバットに当たる、ランニングスロー]', '空三振', '一犠打', '一ゴロ  [詰り]', '右２  [ライン際]', '空三振', '右安  [泳ぎ]', '空三振', '空三振', '中２', '三邪飛', '右安', '右安', '空三振', '中飛', '二ゴロ  [バットの先]', '四球', '四球  [申告敬遠]', '見三振', '右飛', '三ゴロ', '犠打失', '遊飛  [泳ぎ]', '遊飛  [泳ぎ、バットの先、インフィールドフライ]', '見三振  [構えたところにズバリ]', '右飛', '二ゴロ', '空三振  [投手吼える]', '遊併打  [悔しがる]', '四球', '左飛  [バットの先]', '二ゴロ', '遊飛', '空三振', '三犠打  [セーフティバント]', '空三振', '三ゴロ', '中飛  [フェンス激突、ジャンピングキャッチ、好プレイ]', '二飛', '中安', '中安  [投手のグラブを弾く]', '遊併打', '右飛', '遊ゴロ  [バウンド高い]', '三併打  [バットの先、ライン際]', '三ゴロ  [詰り]', '見三振', '投ゴロ', '左飛', '空三振', 'バ三振  [バント空振り]', '左飛', '二ゴロ', '見三振', '一直  [ジャンピングキャッチ、慌てて戻る]', '空三振', '空三振', '一ゴロ  [併殺崩れ]', '四球', '二ゴロ  [ジャンピングスロー]', '空三振', 'ボール  [送球がバウンド、ランエンドヒット]', '三ゴロ  [ライン際、好走塁]', '三ゴロ  [ライン際、好走塁]', '四球', '四球  [帽子が飛ぶ]', '三ゴロ  [バウンド高い]', '左飛', '中飛', '投犠打', '中飛  [タッチアップ]', '空三振  [高めのつり球を要求]', '空三振', '空三

In [15]:
#arrayにはリプレー検証直後のプレーで出塁しているものを入れる
#リプレー検証でイニングや試合が終わっていれば直後のプレーは取りださない
#アウトカウントが変わっていなければ、出塁しているとみなす。
#リプレー検証直後のプレーの出塁率を求める
#cntはリプレー検証直後のプレー数を数える、リプレー検証でイニングや試合が終わっていれば cnt += 1 されない
cnt = 0 #リプレー検証後のプレーの合計
num_obp = 0 #リプレー検証後のプレーのうち出塁した数の合計
array = [] #出塁した結果をまとめた配列
array2 = [] #出塁した結果のindexをまとめた配列
length = df.shape[0] #dfの長さ
for index,data in df.iterrows():
    if ('リプレー検証' in data['結果']) or ('リプレー検証後判定変わらず' in data['結果']) or ('リプレー検証後判定覆る' in data['結果']):
        
        #リプレー検証に関わるプレーで試合が終わっていたら(サヨナラで)飛ばす
        if 'サヨナラ' in data['結果']:
            continue
        #リプレイ検証に関わるプレーでイニングが終了したときの処理
        if df.iat[index,7] == 3: #前のプレーのアウトカウントが3なら(次のイニングに行ったなら):
            continue
        prev = 0
        now = 0
        cnt += 1
        for i in range(index+1,length):
            prev = df.iat[i, 0]

            if i==length-1:
                #もしアウトカウントが変わっていなければ出塁カウンターをプラスして
                #arrayにその内容を加える
                if df.iat[i,7] == df.iat[index,7]:
                    num_obp += 1
                    array.append(df.iat[i,3])
                    print(i)
                    print("-------------")
                    break
                #そうでなければ出塁していないのでbreak
                else:
                    break

            #投球数を見てプレーの切れ目を確認する
            else:
                now = df.iat[i+1,0]
                if prev >= now:
                    #もしアウトカウントが変わっていなければ出塁カウンターをプラスして
                    #arrayにその内容を加える
                    if df.iat[i,7] == df.iat[index,7]:
                        num_obp += 1
                        array.append(df.iat[i,3])
                        array2.append(i)
                        print(i)
                        print(df.iat[index,3])
                        print(df.iat[i,3])
                        print("---")
                        break
                    #そうでなければ出塁していないのでbreak
                    else:
                        break
                        
print(array)
print('リプレー検証後のプレーの合計は'+str(cnt)+'箇所')
print('リプレー検証直後のプレーの出塁した合計は'+str(num_obp)+'箇所')
obp = num_obp / cnt
print('リプレー検証直後のプレーの出塁率は'+str(obp))

167
一安  [バウンド高い、素手で捕球、リプレー検証後判定変わらず]
中２  [フェンス直撃]
---
749
三ゴロ  [併殺崩れ、リプレー検証後判定変わらず]
左２  [泳ぎ、バットの先、内野一塁寄り]
---
1518
遊安  [逆シングル、送球がバウンド、リプレー検証後判定覆る]
四球  [ワンバウンド、ランエンドヒット]
---
3066
ファウル  [ポール際、リプレー検証後判定変わらず]
四球
---
4829
遊ゴロ  [併殺崩れ、交錯、交錯、リプレー検証後判定変わらず]
中２  [目測を誤る]
---
4980
右安  [リプレー検証後判定変わらず]
右２
---
5295
右安  [バットの先、バット折れる、ポテンヒット、リプレー検証後判定変わらず]
中２
---
9590
ボール  [ランエンドヒット、リプレー検証後判定変わらず]
右２  [ライン際]
---
10515
二ゴロ  [併殺崩れ、リプレー検証後判定変わらず]
右安  [泳ぎ]
---
14343
左安  [送球間に進塁、リプレー検証後判定変わらず]
中２
---
15098
ボール  [ワンバウンド、ヘッドスライディング、リプレー検証後判定変わらず]
右安
---
15520
右飛  [リプレー検証後判定変わらず]
右安
---
16177
ボール  [ワンバウンド、ヘッドスライディング、リプレー検証後判定覆る]
四球
---
16675
ボール  [リプレー検証後判定覆る]
四球  [申告敬遠]
---
18783
一犠野  [セーフティスクイズ、グラブトス、リプレー検証後判定変わらず]
犠打失
---
22393
三ゴロ  [送球がバウンド、リプレー検証後判定変わらず]
四球
---
27194
ボール  [送球がバウンド、リプレー検証後判定覆る]
空三振
---
30466
一ゴロ  [併殺崩れ、リプレー検証後判定覆る]
中安
---
31322
三安  [ダイビングキャッチ、リプレー検証後判定変わらず]
中安  [投手のグラブを弾く]
---
36860
空振り  [ランエンドヒット、リプレー検証後判定変わらず]
二ゴロ
---
38242
ファウル  [ライン際、リプレー検証後判定覆る]
空三振
---
38901
ボール  [ワンバウンド、リプレー検証後判定変わら

In [16]:
#26は上のarrayの中で"ボール"や"空三振"など出塁していない要素の数
159-26

133

In [17]:
133/428

0.3107476635514019

In [18]:
print(array2)

[167, 749, 1518, 3066, 4829, 4980, 5295, 9590, 10515, 14343, 15098, 15520, 16177, 16675, 18783, 22393, 27194, 30466, 31322, 36860, 38242, 38901, 40561, 40811, 41553, 46879, 50003, 51726, 55336, 56749, 57564, 63447, 64277, 64535, 64690, 67221, 68128, 72709, 72835, 74438, 76231, 77418, 82023, 82387, 82755, 83603, 85318, 86438, 86720, 87436, 93805, 95044, 95452, 96800, 96863, 98517, 98830, 102283, 102301, 102458, 106772, 107308, 110255, 114393, 117784, 122424, 126379, 128503, 129136, 129902, 132370, 137700, 142986, 144090, 144440, 146590, 149558, 149744, 152741, 153472, 153549, 154651, 154663, 155500, 157773, 158071, 158304, 159334, 159366, 159489, 160269, 161435, 161963, 163853, 166276, 169759, 170761, 174562, 175306, 175577, 175880, 179931, 180210, 181083, 181270, 183536, 184090, 184773, 185070, 186047, 186852, 188742, 190554, 191310, 192815, 193281, 194767, 195377, 195388, 196407, 197658, 198102, 198462, 199082, 201015, 206040, 206717, 208387, 211271, 211433, 212014, 212976, 215636, 21